# Reading In PlaneFlight Output Files from GEOS-Chem

This notebook demonstrates how to use `planeflight_io` to read GEOS-Chem `plane.log` output files into pandas DataFrames or xarray Datasets, either individually or concatenated across multiple days.

In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import os
import planeflight_io as pln

# Set this to the path of the 'examples/' folder in your local clone of the planeflight_io repo.
path_to_examples = '/path/to/planeflight_io/examples'  # <-- Update this!

# Paths to supporting files required by the read functions:
filepath = path_to_examples + '/datafiles_for_examples/'
spdb_yaml = filepath + 'species_database.yml'
config_yaml = filepath + 'geoschem_config.yml'

## Option 1: Read a Single `plane.log` File

`pln.read_planelog()` reads one `plane.log` file and returns:
- **`df`** — a `pd.DataFrame` with one row per observation point and one column per sampled variable.
- **`info_dict`** — a dictionary with metadata for every column in `df`. Each key is a variable name; its value is a dict containing at minimum `'Long_name'` and `'Units'`. Advected species entries also include `'MW_g'` and `'Formula'`.

Two YAML files from your GEOS-Chem run directory are required:

| Argument | File | Why it's needed |
|---|---|---|
| `spdb_yaml` | `species_database.yml` | Provides metadata (molecular weight, formula, long name, units) for every output column. Without it, `info_dict` entries will be incomplete. |
| `config_yaml` | `geoschem_config.yml` | Identifies which output columns are advected tracers vs. other diagnostics, so the correct units can be assigned to each column. |

Both files live in your GEOS-Chem run directory (the directory where you ran GEOS-Chem).

**Units note:** If your input files used tracer *numbers* (recommended default), advected species concentrations will already be in `mol/mol`. If they used tracer *names*, concentrations will be in `molec/cm³` — use `convert2_molmol=True` in Option 2 to convert.

In [ ]:
# outputs_nums/ contains plane.log files generated from input files that used
# tracer numbers — concentrations are already in mol/mol:
df, info_dict = pln.read_planelog(
    filepath + 'outputs_nums/plane.log.20170116',
    spdb_yaml=spdb_yaml,
    config_yaml=config_yaml,
)

# Print a summary of all variable metadata:
for var, meta in info_dict.items():
    print(f"{var:20s}  units={meta.get('Units', 'N/A'):15s}  {meta.get('Long_name', '')}")

## Option 2: Concatenate Multiple `plane.log` Files

`pln.read_and_concat_planelogs()` reads all `plane.log` files in a directory, concatenates them, and optionally saves the result as a NetCDF file.

Key arguments:

| Argument | Description |
|---|---|
| `convert2_molmol` | Set `True` if input files used tracer *names* (outputs are in `molec/cm³`) |
| `as_xarr` | Set `True` to return an xarray Dataset |
| `output_dir` + `output_file` | Where to save the concatenated NetCDF |

**Workflow tip:** Run with `concat=True` once after getting GEOS-Chem output, then set `concat=False` to load the saved file on subsequent runs. This avoids re-reading and re-concatenating every time.

In [ ]:
concat = False  # Set to True to (re-)create the concatenated file

if concat:
    if os.path.isfile(filepath + 'all_PlaneLogs.nc'):
        os.remove(filepath + 'all_PlaneLogs.nc')

    # outputs_names/ used tracer names → concentrations in molec/cm³ → convert:
    ds = pln.read_and_concat_planelogs(
        filepath + 'outputs_names/',
        spdb_yaml=spdb_yaml,
        config_yaml=config_yaml,
        as_xarr=True,
        convert2_molmol=True,
        output_dir=filepath,
        output_file='all_PlaneLogs',
        overwrite=True,
    )
else:
    ds = xr.open_dataset(filepath + 'all_PlaneLogs.nc')

print(ds)

## Plotting the Output

Once you have an xarray Dataset, you can plot model output directly against time or observation data.

In [ ]:
tracer = 'O3'
print('Units of', tracer, ': mol mol\u207b\u00b9 \u00d7 1e9 = ppbv')

plt.plot(ds.time_UTC, ds[tracer] * 1e9, label='Model', linewidth=2)
plt.title(tracer + ' (ppbv)')
plt.xlabel('Time (UTC)')
plt.ylabel('Concentration (ppbv)')
plt.legend()
plt.tight_layout()
plt.show()